### Получение файлов с отзывами для создания датасета на основе генераций с помощью Gemini

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../datasets/csv_for_db_structure/final_csv_files/reviews.csv")
df

,id,site_specific_id,source,date,review_text,source_topic,source_subtopic,rating
0,0,12337288,banki.ru,2025-05-31,Очень давно не пользовалась данным банком так ...,Вклад,NaN,5.0
1,1,12337074,banki.ru,2025-05-31,Сегодня я делал перевод со своей основной карт...,Денежный перевод,NaN,5.0
2,2,12337072,banki.ru,2025-05-31,Добрый день. При оформлении дебетовой карты Га...,Дебетовая карта,NaN,1.0
3,3,12337059,banki.ru,2025-05-31,В конце апреля по моей ссылке отец оформил деб...,Дебетовая карта,NaN,1.0
4,4,12337031,banki.ru,2025-05-31,"Банк ,без объяснения причины, заблокировал в м...",Дебетовая карта,NaN,1.0
...,...,...,...,...,...,...,...,...
25647,25647,11314144,banki.ru,2024-01-01,Оформил дебетовую карту МИР Supreme 12 сентябр...,Дебетовая карта,NaN,1.0
25648,25648,11314037,banki.ru,2024-01-01,Здравствуйте. Хочу оставить отзыв на качество ...,Кредитная карта,NaN,1.0
25649,25649,11313999,banki.ru,2024-01-01,"здравствуйте, вынужден писать здесь, так как п...",Дебетовая карта,NaN,1.0
25650,25650,11313919,banki.ru,2024-01-01,"Сегодня, 1 января, на всех накопительных счета...",Вклад,NaN,2.0


In [5]:
df["source_topic"].value_counts()

source_topic
Дебетовая карта                              15560
Кредитная карта                               3280
Вклад                                         2024
Другое (физ. лица)                            1238
Дистанционное обслуживание физических лиц      798
Денежный перевод                               640
Дебетовые карты                                468
Потребительский кредит                         338
Другие услуги                                  216
Ипотека                                        215
Дистанционное обслуживание                     195
Кредитные карты                                174
Обслуживание                                   151
Вклады                                          93
Мобильное приложение                            87
Автокредит                                      87
Реструктуризация / Рефинансирование             33
Кредиты наличными                               26
Автокредиты                                     12
Обслуживание физич

In [6]:
df_randomised = df.sample(df.shape[0], random_state=42)
df_randomised

,id,site_specific_id,source,date,review_text,source_topic,source_subtopic,rating
2805,2805,12215283,banki.ru,2025-03-24,Я являюсь премиум-клиентом. Сегодня обратилась...,Вклад,NaN,1.0
12519,12519,11808169,banki.ru,2024-10-19,В свое время один сотовый оператор бесплатно н...,Дебетовая карта,NaN,1.0
14015,14015,11764969,banki.ru,2024-10-03,Угораздило связаться. С самого начала заинтере...,Дебетовая карта,NaN,2.0
5252,5252,12132017,banki.ru,2025-02-06,"Здравствуйте, коллеги!\nМои счета были заблоки...",Дебетовая карта,NaN,-1.0
1489,1489,950119,sravni.ru,2025-04-22,Данный банк искусственно затягивает обращение ...,Дистанционное обслуживание,NaN,NaN
...,...,...,...,...,...,...,...,...
21575,21575,11496358,banki.ru,2024-05-05,Являюсь клиентов ГПБ Премиум и решил воспользо...,Дебетовая карта,NaN,2.0
5390,5390,12129210,banki.ru,2025-02-04,Как и у user-12898028606 и у user-324619842424...,Кредитная карта,NaN,2.0
860,860,12293788,banki.ru,2025-05-07,"Здравствуйте, я случайно нажал заблокировать к...",Дебетовая карта,NaN,1.0
15795,15795,11724671,banki.ru,2024-09-05,"Сделал заказ на ozon на, всегда оплачивал чере...",Кредитная карта,NaN,1.0


In [ ]:
chunk_size = 350  # Количество отзывов в одном файле
output_directory = '../../Gemini dataset creation/final_version/input' # Папка для сохранения файлов

In [ ]:
import os

total_rows = df_randomised.shape[0]

for df_start in range(0, total_rows, chunk_size):
    df_end = df_start + chunk_size
    
    # 1. Вырезаем "кусок" (чанк) датафрейма
    df_part = df_randomised.iloc[df_start:df_end].copy()

    # 2. Убираем переносы строк внутри текста отзыва
    df_part['review_text'] = df_part['review_text'].str.replace('\n', ' ').str.replace('\r', ' ')

    # 3. Форматируем каждую строку в нужный вид
    texts_series = "Отзыв " + df_part['id'].astype(str) + ": " + df_part['review_text']

    # 4. Объединяем все отформатированные строки в один большой текст
    combined_text = "\n\n".join(texts_series.values)

    # 5. Сохраняем в файл с динамическим именем
    file_path = os.path.join(output_directory, f'gemini_{df_start}-{df_start+len(df_part)}.txt')
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(combined_text)
        
    print(f"Сохранен файл: {file_path} ({len(df_part)} отзывов)")

Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_0-350.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_350-700.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_700-1050.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_1050-1400.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_1400-1750.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_1750-2100.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_2100-2450.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_2450-2800.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_2800-3150.txt (350 отзывов)
Сохранен файл: ../../Gemini dataset creation/final_version/input\gemini_3150-3500.txt (350 отзывов)
Сохране

### Промпты для генерации

#### Для Gemini

Твоя задача — проанализировать отзывы клиентов Газпромбанка (ГПБ). Для каждого отзыва тебе необходимо:
1.  Составить краткую текстовую сводку (summarized_review).
2.  Определить все продукты/услуги (темы), о которых упоминает клиент.
3.  Для каждой упомянутой темы определить тональность высказывания: positive, negative или neutral.

**ВАЖНЫЕ ИНСТРУКЦИИ:**
-   **Темы:** В основном используй следующий список: Дебетовые карты, Офисное обслуживание, Дистанционное обслуживание, Кредиты наличными, Кредитные карты, Вклады, Ипотека, Автокредиты, Реструктуризация кредитов, Рефинансирование кредитов, Рефинансирование ипотеки, Обмен валют, Мобильное приложение, Потребительский кредит. Если в отзыве описана другая тема, ты можешь добавить ее. Помимо этого, можешь указывать уточняющие темы, такие как продукты банка, которые приведены ниже.
-   **Тональность:** `neutral` указывается тогда, когда тема упомянута как факт, без какой-либо эмоциональной окраски (ни положительной, ни отрицательной).
-   **Строгость:** Не выдумывай темы. Если в отзыве нет явного упоминания продукта или услуги, не включай его.
-   **Символы `****`:** Это либо конфиденциальные данные (номера телефонов), либо ненормативная лексика. Учитывай общий контекст вокруг них для определения тональности.
-   **Если темы нет:** Если в отзыве невозможно определить ни одну тему, верни пустой массив `topic_sentiment_pairs`.

Вот список всех продуктов банка на сегодняшний день:
* **дебетовые карты:**
    * Умная дебетовая карта «Мир»
    * Премиальная карта Mir Supreme
    * Дебетовая карта с кэшбэком для самозанятых
    * Умная дебетовая карта «Мир»
    * Карта для автолюбителей «Газпромбанк—Газпромнефть»
    * Виртуальная дебетовая карта ГПБ&ФК «Зенит»
    * Дебетовая Пенсионная карта
* **Кредитные карты:**
    * Кредитная карта с льготным периодом до 120 дней
    * Простая кредитная карта
    * Кредитная карта 90 дней
    * Кредитная карта 180 дней Премиум
    * Кредитная карта для самозанятых
* **Накопительные счета:**
    * Накопительный счет
    * «Ежедневная выгода»
    * «Ежедневный процент»
    * «Премиум»
    * Социальный счет
* **Вклады:**
    * Вклад «Новые деньги»
    * Вклад «Ключевой момент»
    * Вклад «Копить»
    * Вклад «В Плюсе»
    * Вклад «Расширяй возможности»
    * Социальный вклад
* **Кредиты:**
    * Кредит наличными
    * Кредит наличными под залог недвижимости
    * Кредит на авто и другие цели
    * Рефинансирование потребительских кредитов
    * Дачный кредит
    * Кредит на образование
    * Кредит наличными для бюджетников
* **Другие услуги банка:**
    * Газпромбанк Мобайл
    * Газпромбанк Travel
    * Gazprom Pay
    * GorodPay
    * Газпром Бонус
    * Страховые и сервисные продукты
    * Депозитарные услуги


**Формат ответа — строго JSON:**
[
    {
        "id" : "исходный_id_отзыва",
        "summarized_review": "Краткая сводка отзыва на русском языке.",
        "topic_sentiment_pairs": [
            {
                "topic": "Название темы 1",
                "sentiment": "positive/negative/neutral"
            },
            {
                "topic": "Название темы 2",
                "sentiment": "positive/negative/neutral"
            }
        ]
    }
]


**Пример:**
Отзыв 1: Мобильное приложение просто ужасное, постоянно вылетает. А вот вклад оформил недавно — условия хорошие.


Отзыв 2: Звонил узнать насчет ипотеки, но в поддержке ничем не помогли.

Ответ:
[
    {
        "id" : 1,
        "summarized_review": "Клиент критикует мобильное приложение за нестабильность, но хвалит условия по вкладу.",
        "topic_sentiment_pairs": [
            {
                "topic": "Мобильное приложение",
                "sentiment": "negative"
            },
            {
                "topic": "Вклады",
                "sentiment": "positive"
            }
        ]
    },
    {
        "id" : 2,
        "summarized_review": "Поддержка не смогла помочь клиенту по вопросу с ипотекой.",
        "topic_sentiment_pairs": [
            {
                "topic": "Дистанционное обслуживание",
                "sentiment": "negative"
            },
            {
                "topic": "Ипотека",
                "sentiment": "neutral"
            }
        ]
    },
]

**Вот отзывы клиентов, которые необходимо обработать:**

#### Для обучаемых моделей

Твоя задача — проанализировать отзывы клиентов Газпромбанка (ГПБ). Для каждого отзыва тебе необходимо:
1.  Определить все продукты/услуги (темы), о которых упоминает клиент.
2.  Для каждой упомянутой темы определить тональность высказывания: positive, negative или neutral.

**ВАЖНЫЕ ИНСТРУКЦИИ:**
-   **Темы:** В основном используй следующий список: Дебетовые карты, Офисное обслуживание, Дистанционное обслуживание, Кредиты наличными, Кредитные карты, Вклады, Ипотека, Автокредиты, Реструктуризация кредитов, Рефинансирование кредитов, Рефинансирование ипотеки, Обмен валют, Мобильное приложение, Потребительский кредит. Если в отзыве описана другая тема, ты можешь добавить ее. Помимо этого, можешь указывать уточняющие темы, такие как продукты банка, которые приведены ниже.
-   **Тональность:** `neutral` указывается тогда, когда тема упомянута как факт, без какой-либо эмоциональной окраски (ни положительной, ни отрицательной).
-   **Строгость:** Не выдумывай темы. Если в отзыве нет явного упоминания продукта или услуги, не включай его.
-   **Символы `****`:** Это либо конфиденциальные данные (номера телефонов), либо ненормативная лексика. Учитывай общий контекст вокруг них для определения тональности.
-   **Если темы нет:** Если в отзыве невозможно определить ни одну тему, верни пустой массив `topic_sentiment_pairs`.

Вот список всех продуктов банка на сегодняшний день:
* **дебетовые карты:**
    * Умная дебетовая карта «Мир»
    * Премиальная карта Mir Supreme
    * Дебетовая карта с кэшбэком для самозанятых
    * Умная дебетовая карта «Мир»
    * Карта для автолюбителей «Газпромбанк—Газпромнефть»
    * Виртуальная дебетовая карта ГПБ&ФК «Зенит»
    * Дебетовая Пенсионная карта
* **Кредитные карты:**
    * Кредитная карта с льготным периодом до 120 дней
    * Простая кредитная карта
    * Кредитная карта 90 дней
    * Кредитная карта 180 дней Премиум
    * Кредитная карта для самозанятых
* **Накопительные счета:**
    * Накопительный счет
    * «Ежедневная выгода»
    * «Ежедневный процент»
    * «Премиум»
    * Социальный счет
* **Вклады:**
    * Вклад «Новые деньги»
    * Вклад «Ключевой момент»
    * Вклад «Копить»
    * Вклад «В Плюсе»
    * Вклад «Расширяй возможности»
    * Социальный вклад
* **Кредиты:**
    * Кредит наличными
    * Кредит наличными под залог недвижимости
    * Кредит на авто и другие цели
    * Рефинансирование потребительских кредитов
    * Дачный кредит
    * Кредит на образование
    * Кредит наличными для бюджетников
* **Другие услуги банка:**
    * Газпромбанк Мобайл
    * Газпромбанк Travel
    * Gazprom Pay
    * GorodPay
    * Газпром Бонус
    * Страховые и сервисные продукты
    * Депозитарные услуги


**Формат ответа — строго JSON:**
{
    "id" : "исходный_id_отзыва",
    "topic_sentiment_pairs": [
        {
            "topic": "Название темы 1",
            "sentiment": "positive/negative/neutral"
        },
        {
            "topic": "Название темы 2",
            "sentiment": "positive/negative/neutral"
        }
    ]
}


**Примеры:**
Отзыв 1: Мобильное приложение просто ужасное, постоянно вылетает. А вот вклад оформил недавно — условия хорошие.

Ответ:
{
    "id" : 1,
    "topic_sentiment_pairs": [
        {
            "topic": "Мобильное приложение",
            "sentiment": "negative"
        },
        {
            "topic": "Вклады",
            "sentiment": "positive"
        }
    ]
}


Отзыв 2: Звонил узнать насчет ипотеки, но в поддержке ничем не помогли.

Ответ:
{
    "id" : 2,
    "topic_sentiment_pairs": [
        {
            "topic": "Дистанционное обслуживание",
            "sentiment": "negative"
        },
        {
            "topic": "Ипотека",
            "sentiment": "neutral"
        }
    ]
},

**Вот отзывы клиентов, которые необходимо обработать:**